
# Image classification with a Convolutional Neural Network

## Malaria cell classification

Malaria is a blood disease caused by the *Plasmodium* parasite, and is transmitted through the bite of the female *Anopheles* mosquito. The disease is mostly diagnosed by counting parasitized blood cells in a blood smear under a microscope. However, manual cell counting is an exhausting, error-prone procedure. This can negatively affect the quality of the diagnosis [[1]](https://peerj.com/articles/4568/). Especially in *resource-constrained* regions of the world, difficult working conditions lead to poor diagnosis quality [[2]](https://lhncbc.nlm.nih.gov/publication/pub9932) .

In this practical session we will develop a deep learning pipeline that will aid in improving malaria diagnosis by automating infected cell counting. To this end we will use the Malaria cell dataset [[2]](https://lhncbc.nlm.nih.gov/publication/pub9932) to train a neural network that predicts a cell's infection state based on a microscopy image of it. The microscopy images were acquired using a smartphone attached to a small portable microscope.

To set up the deep learning pipeline we will go through these steps:
- Load the dataset
- Preprocess the dataset
- Define a neural network
- Define the learning procedure
- Train the neural network
- Inspect the network's performance

We will use the [TensorFlow machine learning framework](http://tensorflow.org) to implement these steps.

# Setup

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
tf.random.set_seed(42)

In [ ]:
from tensorflow.python.client import device_lib

def select_device(prefer_gpu=True):
    local_device_protos = device_lib.list_local_devices()
    gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
    if (len(gpus) > 0) and prefer_gpu:
        return gpus[0]
    else:
        return [x.name for x in local_device_protos if x.device_type == 'CPU'][0]

In [ ]:
# This code sets the device to use to GPU if you have one available
device = select_device(prefer_gpu=True)
device

The above `device` is used in the notebook to select where code is executed. This is done with a `with`-statement:
```
with tf.device(device):
    # code
```

# Load the Malaria dataset

We use the `tensorflow-datasets` package to load the malaria dataset:

In [ ]:
# The dataset has been recently moved and tensorflow-datasets hasn't been updated yet,
# so we update the URL manually
tfds.image_classification.malaria._URL="https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip"

# Now we can create the builder
builder = tfds.builder('malaria', data_dir="C:\data")

In [ ]:
builder.download_and_prepare()

Look at some info about the dataset.

In [ ]:
builder.info

Use the builder to extract a TensorFlow Dataset for efficient access to the images.
We also split the dataset into training data for training the neural network parameters, and testing data for evaluating the network's performance on unseen data.

In [ ]:
with tf.device(device):
    train_ds, test_ds = (
        builder.as_dataset(as_supervised=True, split="train[:80%]"),
        builder.as_dataset(as_supervised=True, split="train[-20%:]")
    )

**TASK 1** 

Plot some images from `train_ds` to get an idea of what the data looks like. Have a look at the [TensorFlow Datasets documentation](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) to see how you can *take* a number of images from the dataset.

# Preprocess the dataset

For this dataset two preprocessing operations are required:
1. Resize all images to uniform width and height, and
2. normalize the pixel value range to [0,1].

Neural networks have a fixed architecture and can therefore only take inputs of equal dimensions. This is why we have to preprocess the images to have uniform dimensions. For the Malaria dataset, we downsample the images to be 40 pixels high and 40 pixels wide. We will need the `resize_with_pad` and `resize` functions from the `tf.image` module.

Normalization of the pixel range to a [0,1] range is done to improve the stability of the weight updates. The Malaria dataset contains 8-bit RGB images, which have a maximum pixel value of 255 and a minimum of 0. If we would train the neural network using pixel values in the [0,255] range, the network's weights could grow too large causing high values during backpropagation and unstable training.

In [ ]:
# define the first preprocessing function
# note: in this function we also cast the images to the float32 data type.

image_width, image_height = 40, 40
def resize_images(image, label):
    return (
        tf.cast(tf.image.resize_with_pad(tf.image.resize(image, (image_width, image_height)), image_width, image_height), tf.float32), 
        tf.cast(label, tf.float32)
    )

In [ ]:
# define the second preprocessing function
def minmax_normalization(image, label):
    return (
        image / tf.math.reduce_max(tf.reshape(image, [-1, image.shape[-1]]), axis=0), # divide each pixel in the image by the maximum value in each channel (R, G, and B)
        label
    )

**TASK 2** 

Apply both preprocessing functions to the train and test dataset defined earlier. Have a look at the [TensorFlow Datasets documentation](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) to see how you can *map* a function over the elements of a dataset.

The data is now preprocessed and ready to handled by the neural network that we will define in the next part.

# Define a neural network architecture for image classification

We need a neural network that is capable of classifying cell images in a positive and negative class. In this case, we want it to distinguishing between a healthy and parasitized cell based on a 40x40 RGB image. For today's standards this specific problem is a fairly easy computer vision task. A simple (and efficient) _shallow_ convolutional neural network (CNN) will suffice.

## CNN Architecture

The neural network will consist out of 4 main layers: 2 feature extraction layers, and 2 classification layers.

### Feature extraction

The feature extraction layers will learn to extract relevant features from the image. These features enable the classification layers to learn to classify cells into the healthy or parasitized category. The feature extraction layers are made up of convolution and max-pooling operations, and activation functions.

![Convolution](images/convolution_overview.gif "convolution")
![Max pooling](images/maxpool_animation.gif "maxpool")

(from https://developers.google.com/machine-learning/practica/image-classification/convolutional-neural-networks?hl=ru)

### Classification 

The extracted features get passed on to the classification layers. These are made up of densely connected layers, dropout connections and activation functions.

![Dropout](images/dropout.gif "dropout")

(from https://nagadakos.github.io/2018/09/23/dropout-effect-discussion/)

### Activation functions

The activation functions used in this neural network are the sigmoid, and Rectified Linear Unit (ReLU). 
![ReLU](images/relu.png "ReLU")

The ReLU is used as the intermediate activation function in the feature extraction and classification layers. This function is used in many state-of-the-art image classification networks. It works well because it prevents the gradients from *vanishing* during backpropagation. The SELU activation is also frequently used, and adds some theoretical convergence guarantees.

![Sigmoid](images/sigmoid.png "Sigmoid")

The sigmoid is the activation function applied to the output of the final densely connected layer. It squeezes whatever value comes out of the network to the 0-1 range. Ideal, for binary classification!

## Architecture diagram

![SegmentLocal](images/network-page-001.jpg "Network diagram")

**TASK 3** 

Define a CNN architecture with the [TensorFlow Keras API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) using the above scheme as a guideline. Feel free to try out some of your own settings! The network takes RGB images as input, and has one output neuron that will produce output values between 0 (malaria) and 1 (healthy).

You will need the following layers:
- `tf.keras.layers.Conv2D`
- `tf.keras.layers.MaxPool2D`
- `tf.keras.layers.ReLU`
- `tf.keras.layers.Flatten`
- `tf.keras.layers.Dense`
- `tf.keras.layers.Dropout`

# Define the loss function, optimizer, and metrics

To optimize the parameters of the network we need an optimization algorithm, and a loss function to minize. In this particular case the binary cross-entropy is a good choice.

In [ ]:
bce_loss = tf.keras.losses.BinaryCrossentropy()

 As optimization algorithm we choose stochastic gradient descent (SGD). More advanced optimizers exist, but SGD is a good first choice.

In [ ]:
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

The binary cross-entropy is a good metric for optimization, but is less interpretable. Accuracy is a more intuitive metric for assessing model performance.

In [ ]:
with tf.device(device):
    accuracy_metric = tf.keras.metrics.BinaryAccuracy()

Finally, TensorFlow requires us to `compile` all parts of the learning procedure together.

In [ ]:
with tf.device(device):
    classifier.compile(optimizer=sgd_optimizer, loss=bce_loss, metrics=[accuracy_metric])

# Run learning procedure to train the network's parameters

First, we define some constants for the learning procedure.

In [ ]:
epochs = 30 # How many times will the network see all training data
batch_size = 256 # How many instances will the network process in one iteration

train_len = int(builder.info.splits["train"].num_examples*0.8)
test_len = int(builder.info.splits["train"].num_examples*0.2)
steps_per_epoch = train_len//batch_size
test_steps = test_len//batch_size

Next, we iteratively update the parameters of the network using mini-batches of data (of size `batch_size`). In other words, we fit the network to the data. In pseudo-code:
```
for epoch in epochs:
    for true_labels, batch in batches:
        predictions = network(batch)
        loss = loss_function(true_labels, predictions)
        optimizer.update(network, loss)
```
The TensorFlow Keras API has several utitility functions that wrap basic procedures in helper functions. One such function is `fit` ([documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit)). It performs something close to the above pseudo-code on our network. All we have to do, is pass it the constants we defined above. Using helper functions is good practice as it lets us write code quicker, and it is less error-prone. However, always know what the helper functions do exactly!

Note that we save the return value of the fit function to a variable. It contains important information about the performance of our network. We will analyze this in the next part.

In [ ]:
with tf.device(device):
    history = classifier.fit(
        train_ds.batch(batch_size).repeat(), # on which data to we want to train
        epochs=30, # how many epochs do we want to run
        steps_per_epoch=steps_per_epoch, # how many steps are in one epoch
        validation_data=test_ds.batch(batch_size).repeat(), # what test data do we want to use
        validation_steps=test_steps # how many steps do we need to take when testing
    )

**TASK 5**

Before doing this task, go to [Inspect classifier performance](#Inspect-classifier-performance). Afterwards, return to this task.

For now, we have used the SGD optimizer. Many other optimizers exist, such as the Adam optimizer. This optimizer combines two extensions of SGD into one powerful optimizer. Try to use the Adam optimizer to fit the neural network. See how the training progresses, do you notice any difference compared to SGD?

**NOTE**: In order to retrain the classifier with another optimizer you have to reinstantiate the classifier.

# Inspect classifier performance

After fitting the network, we are interested in its eventual performance and how this evolved during the fitting procedure. By comparing metrics computed on the training and test dataset we can spot overfitting as well: if the training accuracy is much higher compared to testing, the network is overfitting.

The information we need, is returned by the fit function in a [`History`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History) object.

**TASK 6** 

Plot the the accuracy and loss in function of the epochs for the train and validation set. Use the history object, returned by the `fit` function.

**TASK 6**

Use the trained network to compute prediction scores for the test dataset. Use these scores to compute the confusion matrix and ROC AUC score.

# Extra: A look at intermediate representations of the network

As an image passes through our convolutional neural network, it is transformed by the operations defined in the layers. Each transformation extracts the most relevant information from the input it receives, and passes it on to the next layer. The output of each intermediate layer is called a feature map or *representation*.

By recording these intermediate representations for an image, we get an idea of what the network is focusing on to make its prediction. This way, we get some insight into the so-called *black-box model*.

In this last part we will look at some of the representations learned by the network we trained. Before running the code, think about what these representations might look like.

In [ ]:
# extract some images to analyse
n_images = 10
it = iter(test_ds.batch(n_images))
images, labels = next(it)

# get the predictions
predictions = classifier.predict(images)

# record the intermediate representations
representations = []
record = [1, 2, 3]
x = images
for i, layer in enumerate(classifier.layers):
    x = layer(x)
    if i in record:
        representations.append(x)

In [ ]:
fig, axes = plt.subplots(1+len(representations), len(images), figsize=(40, 10), constrained_layout=True)
    
for ax,img,label,prediction in zip(axes[0, :], images, labels, predictions):
    ax.imshow(img)
    txt_label = "Healthy" if label.numpy() == 1 else "Malaria"
    txt_pred = "Healthy" if prediction > 0.5 else "Malaria"
    ax.set_title(f"{txt_label}, ({txt_pred})")
    
for reps, row in zip(representations, axes[1:, :]):
    for rep, ax in zip(reps, row):
        ax.imshow(rep[..., 0])